# Advanced Topic: Using External C++ Functions

This is based on the relevant portion of the CmdStan documentation [here](https://mc-stan.org/docs/cmdstan-guide/using-external-cpp-code.html)

Consider the following Stan model, based on the bernoulli example.

In [ ]:
import os
from cmdstanpy import cmdstan_path, CmdStanModel
model_external = CmdStanModel(stan_file='bernoulli_external.stan', compile=False)
print(model_external.code())

As you can see, it features a function declaration for `make_odds`, but no definition. If we try to compile this, we will get an error. 

In [ ]:
model_external.compile()

Even enabling the `--allow_undefined` flag to stanc3 will not allow this model to be compiled quite yet.

In [ ]:
model_external.compile(stanc_options={'allow_undefined':True})

To resolve this, we need to both tell the Stan compiler an undefined function is okay **and** let C++ know what it should be. 

We can provide a definition in a C++ header file by using the `user_header` argument to either the CmdStanModel constructor or the `compile` method. 

This will enables the `allow_undefined` flag automatically.

In [ ]:
model_external.compile(user_header='make_odds.hpp')

We can then run this model and inspect the output

In [ ]:
fit = model_external.sample(data={'N':10, 'y':[0,1,0,0,0,0,0,0,0,1]})
fit.stan_variable('odds')

The contents of this header file are a bit complicated unless you are familiar with the C++ internals of Stan, so they are presented without comment:

```c++
#include <boost/math/tools/promotion.hpp>
#include <ostream>

namespace bernoulli_model_namespace {
    template <typename T0__>  inline  typename
          boost::math::tools::promote_args<T0__>::type  
          make_odds(const T0__& theta, std::ostream* pstream__) {
            return theta / (1 - theta);  
       }
}
```